<a href="https://colab.research.google.com/github/daliarod96/DAPT-MLM-BERT/blob/main/pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Transformers library
# I'm using an old version because newer versions were not compatible with
# the Docker container that I used to run this program
!pip install transformers==2.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 40.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=66a5648ca15a8d5c21b396af72030ff7b5e581950a66079011eb040ba5d32665
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [2]:
# Import necessary libraries 
import pandas as pd 
import torch
import torch.nn as nn
import transformers

In [3]:
# creat a custom Tweets Dataset class 
class TweetsDataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings['input_ids'])


In [7]:
# Import and process dataset
text = pd.read_csv("/content/drive/MyDrive/myModels/profanitydatasetprocessed")
text = text['content']
text = text.dropna()
text = text.astype('str')
text = text.tolist()
text = text[:100]


In [8]:
# create the tokenizer and load the pre-trained model

from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", do_lower_case=True)
print("Tokenizer imported")
model = BertForMaskedLM.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
print("Model imported")


Tokenizer imported
Model imported


In [9]:
# tokenize text for training

result = tokenizer.batch_encode_plus(text)
chunk_size = 512

def group_texts(examples):
  # concatenate all texts
  concatenated_examples = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
  for i in range(len(examples['input_ids'])):
    concatenated_examples['input_ids']+=examples['input_ids'][i]
  for i in range(len(examples['token_type_ids'])):
    concatenated_examples['token_type_ids']+=examples['token_type_ids'][i]
  for i in range(len(examples['attention_mask'])):
    concatenated_examples['attention_mask']+=examples['attention_mask'][i]

  # compute length of concatenated texts
  total_length = len(concatenated_examples['input_ids'])

  # We drop the last chunk if it's smaller than chunk_size
  total_length = (total_length // chunk_size) * chunk_size

  # Split by chunks of max_len
  result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }

  # Create a new labels column
  result["labels"] = result["input_ids"].copy()


  result['input_ids'] = torch.tensor(result['input_ids'])
  result['token_type_ids'] = torch.tensor(result['token_type_ids'])
  result['attention_mask'] = torch.tensor(result['attention_mask'])
  result['labels'] = torch.tensor(result['labels'])

  
  return result

inputs = group_texts(result)




"""
We want to create our mask. 
Each token that is not a special token has a 15% chance of being masked.
We don't want to mask [CLS], [SEP], or padding tokens. 
Correspond to the numbers 4, 5, and 1.
"""

rand = torch.rand(inputs['input_ids'].shape)

mask_arr = (rand < 0.15) * (inputs['input_ids'] != 4) * (inputs['input_ids'] != 5)* (inputs['input_ids'] != 1)

# masked tokens

selection = []

for i in range(mask_arr.shape[0]):
  selection.append(
    torch.flatten(mask_arr[i].nonzero()).tolist()
  )

for i in range(mask_arr.shape[0]):
  inputs['input_ids'][i,selection[i]] = 0

dataset = TweetsDataset(inputs)

# pass dataset into DataLoader
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

model.train()

from torch.optim import AdamW

#optimizer
optim = AdamW(model.parameters(), lr=5e-5, weight_decay = 0.01)

dataset
dataloader
device
model to device


In [11]:
# Training loop

from tqdm import tqdm

epochs = 1

for epoch in range(epochs):
  loop = tqdm(dataloader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels= batch['labels'].to(device)
    

    outputs = model(input_ids, attention_mask=attention_mask,
                   lm_labels=labels)
    loss, prediction_scores = outputs[:2]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-3-f45e2a3e1b39>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 100%|██████████| 1/1 [00:03<00:00,  3.36s/it, loss=0.324]


In [14]:
save_directory = "/content/drive/MyDrive/myModels"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)